In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tiff
import os
from scipy import signal
plt.style.use('default')

# STA

# utils

In [ ]:
def window(curve, k, R1, R2):
    window = curve[k-R1:k+R2]
    return window

def STA1(spikelist, channel, plot, Normalize, peaktime = False): 
    # spike frame list, int Channel, True/False, True/False
    
    R1 = window_size1
    R2 = window_size2
    
    chidx = np.where(Ch_list==channel)[0]
    
    C = DffROI[:,chidx].flatten()

    peaks = np.zeros((R1+R2, len(spikelist)))
    
    
    for idx in range(len(spikelist)):
        
        peaks[:, idx] = window(C, spikelist[idx], R1, R2)

    sta = np.mean(peaks, axis = 1)
    
    if Normalize == True:
        
        sta = sta / np.max(sta)
    
    if plot == True:
        plt.plot(sta, label = "ROI" + str(channel))
    
    if peaktime == True:
        print("peaktime of channel ",channel," => ", np.argmax(sta))
    
    return sta

def STA2(spikelist, channel, plot, Normalize, peaktime = False): 
    # spike frame list, int Channel, True/False, True/False
    
    R1 = window_size1
    R2 = window_size2
    
    chidx = np.where(Ch_list==channel)[0]
    
    C = DffROI[:,chidx].flatten()

    peaks = np.zeros((R1+R2, len(spikelist)))
    
    
    for idx in range(len(spikelist)):
        
        peaks[:, idx] = window(C, spikelist[idx], R1, R2)
        
    if plot:
        plt.plot(peaks)
    
    return peaks

## 1. load file

In [ ]:
#**analyzing with saved DFFfile**

import numpy as np
import matplotlib.pyplot as plt
import tifffile as tiff



###input here###

main_path = "C:/Users/NPL/KGH/230609_Estim_sample"
filepath = "C:/Users/NPL/KGH/230609_Estim_sample/Dff.tif"

###input here###



Dff = tiff.imread(filepath)

numFrames = max(Dff.shape)

projection = np.mean(Dff, axis = 0)

th_idx = np.where(projection != 0)

k = np.array((th_idx[0], th_idx[1]))

ROI = tuple(map(tuple, k.T))

print("total", len(ROI), " pixels available : \n")
print(ROI)

# ROI(Channel) based Analysis

## 2.Set Grid
#### if use already generated grid, go to 2.2

In [ ]:
#manual grid generation


### input here ###

row_0 = 13,28,40,57,69,80,111
row_1 = 14,22,40,80,99,113
row_2 = 18,33,40,57,68,80,104,113
name_0 = 71,72,73,81,82,83,91,92
name_1 = 41,42,43,0,61,62,63
name_2 = 11,12,13, 21, 22, 23, 31,32,33

### input here ###

grid = np.zeros((3,120)).astype('int')

totChannel = len(row_0) + len(row_1) + len(row_2) + 3

count = 0
for k in range (120):
    if k in row_0:
        count = count+1
    grid[0,k] = name_0[count]
    
count = 0    
for k in range (120):
    if k in row_1:
        count = count+1
    grid[1,k] = name_1[count]
    
count = 0    
for k in range (120):
    if k in row_2:
        count = count+1
    grid[2,k] = name_2[count]
    
plt.imshow(grid);

### 2.1 save grid

In [ ]:
# to save grid
tiff.imsave(main_path + "/" + "grid.tif", grid) 

### 2.2 load grid

In [ ]:
#else, load new grid
grid = tiff.imread("C:/Users/NPL/KGH/Spon_denoised/grid.tiff")
plt.imshow(grid)

## 3. Assign pixels to grid(Channel)

In [ ]:
ch_assign = []

for roi in ROI:
    ch_assign.append(grid[roi[0], roi[1]])

Uni = np.unique(ch_assign, return_counts = True)
Ch_list = Uni[0]

print("List of Channel", Ch_list, '\n')

numCH = len(Ch_list)
DffROI = np.zeros((numFrames,numCH))

print("Channel assignment \n")

for idx1 in range(numCH):
    
    channel = Ch_list[idx1]
    
    print(channel)
    
    k = np.where(ch_assign == channel) # index of ROI in certain channel
    
    for idx2 in k[0]:
        coor = ROI[idx2]
        print(coor) # ROI[idx2] = coordinate in certain channel
        
        DffROI[:,idx1] = DffROI[:,idx1] + Dff[:,coor[0], coor[1]]
        
    DffROI[:,idx1] = DffROI[:,idx1] / len(k[0])
    
    
np.savetxt(main_path + "/" + "grid_dff.csv", DffROI, delimiter=',') 

# Spike Triggered Averaging

## Get spike timing(trigger, measured)

In [ ]:
spikelist = [948, 2043, 3138, 4232, 5327, 6422, 7517, 8611, 9706, 10801, 11896, 12990, 14085, 15180, 16275, 17369, 18464, 19559, 20654, 21748, 22843, 23938, 25033, 26127, 27222, 28317, 29412, 30506, 31601, 32696, 33791, 34885, 35980, 37075, 38170, 39264, 40359, 41454, 42549, 43643, 44738, 45833, 46928, 48022, 49117, 50212, 51307, 52401, 53496, 54591, 55686, 56780, 57875, 58970, 60065, 61159, 62254, 63349, 64444, 65538, 66633, 67728, 68823, 69917, 71012, 72107, 73202, 74296, 75391, 76486, 77580, 78675, 79770, 80865, 81959, 83054, 84149, 85244, 86338, 87433]

## 4. Visualize sample ROI and spike timing

In [ ]:
STD_ROI = 11

threshold = 0.13

window_size1 = 50
window_size2 = 350

VIEW_XLINE = True


chidx = np.where(Ch_list==STD_ROI)[0]


STD = DffROI[:,chidx]
STD = STD.flatten()
plt.figure(figsize=(10,3))
plt.plot(STD)


num_spike = len(spikelist)


print("total " + str(num_spike) + " spikes in ROI")
if VIEW_XLINE:
    for spike in spikelist:
        plt.axvline(x = spike, color = 'r', linewidth = 1, alpha = 0.3)

print(spikelist)

## 5. Do STA and TTX artifact removal

In [ ]:
sta_for_std = STA1(spikelist,11, True, False, peaktime = True); #view averaged 

In [ ]:
STA2(spikelist,11, True, True, peaktime = True); #view individual

In [ ]:
for ch in Ch_list: # save individual peaks for all ROIs
    np.savetxt(main_path +"/" + str(ch)+".csv", STA2(spikelist,ch, True, False, peaktime = True), delimiter = ',')

In [ ]:
#ttx artifact removal
ROIAA = 33
ttx = np.genfromtxt(main_path+"/"+str(ROIAA)+"_ttx.csv", dtype = 'float', delimiter = ',')
ttx_avg = np.mean(ttx, axis = 1)
plt.plot(ttx_avg)

In [ ]:
removed = sta_for_std - ttx_avg
plt.plot(removed)
np.savetxt(main_path + "/" + str(ROIAA) + "ttx_removed_avg.csv", removed)